# HandsOn
タイタニックデータを題材に、機械学習モデル作成のグループワークをしてもらいます。<br>
- データの加工
- データの分析・可視化
- モデル作成
  - 変数選択
  - アルゴリズム選択
  - パラメータチューニング
- テストデータ予測
  - submit

機械学習モデル作成の一連の流れを通して面白さ・難しさを体験してみてください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


終了後、各グループ↓この辺りを発表して頂けると面白いかなと思ってます。

- kaggleでsubmitした結果のスコア
- 使用した変数・アルゴリズム・理由など
- 工夫したポイント（特徴量作成など）

## 今回取り組んだ機械学習モデル
料金・客室クラスを特徴量としたロジスティック判別モデル

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data frameの読み込み
df_train = pd.read_csv('/content/drive/MyDrive/data/titanic_train.csv')
df_test = pd.read_csv("/content/drive/MyDrive/data/titanic_test.csv")

In [ ]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df_train.isnull().sum()
#FareとPclassの欠損値はなし！

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(data_frame=df_train, x='Fare', color='Survived', barmode='overlay')

In [ ]:
px.histogram(data_frame=df_train, x='Pclass', color='Survived', barnorm='percent')

In [20]:
df_test['Fare'].mean()

35.6271884892086

In [28]:
df_test['Fare_filled'] = df_test['Fare'].fillna(df_test['Fare'].mean())
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fare_filled
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,7.8292
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,7.0000
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,9.6875
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,8.6625
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,12.2875
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,8.0500
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,108.9000
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,7.2500
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,8.0500


In [36]:
df_train['Fare_filled'] = df_train['Fare']

In [31]:
df_test['Fare_filled'].isnull().sum()

0

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
lr = LogisticRegression()

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [38]:
lis_feature_names = ['Fare_filled', 'Pclass']
y_name = 'Survived'

In [39]:
X_train = scaler.fit_transform(df_train[lis_feature_names])
X_test = scaler.transform(df_test[lis_feature_names])

In [17]:
lr.fit(X_train, df_train[y_name])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
pred = lr.predict(X_test)

In [41]:
df_submit = df_test['PassengerId'].reset_index()
df_submit['Survived'] = pred
df_submit = df_submit.drop('index', axis=1)

In [43]:
df_submit['Survived'].value_counts()

0    311
1    107
Name: Survived, dtype: int64

In [45]:
df_submit.to_csv('Fare&Pclass.csv', index=None)